<a href="https://colab.research.google.com/github/felixbmuller/nlp-commonsense/blob/main/NLP_Commonsense_Assignment_2_Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Commonsense Assignment 2 - Baseline Model

## Setup

In [1]:
!pip install -q transformers datasets torch torchvision

     |████████████████████████████████| 3.5 MB 5.5 MB/s 
     |████████████████████████████████| 311 kB 37.6 MB/s 
     |████████████████████████████████| 895 kB 33.6 MB/s 
     |████████████████████████████████| 6.5 MB 38.6 MB/s 
     |████████████████████████████████| 596 kB 32.1 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 37.5 MB/s 
     |████████████████████████████████| 212 kB 51.0 MB/s 
     |████████████████████████████████| 134 kB 55.7 MB/s 
     |████████████████████████████████| 271 kB 46.4 MB/s 
     |████████████████████████████████| 144 kB 50.1 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
!apt install git-lfs >/dev/null

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.16.2


Set configuration

In [5]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Load and investigate the dataset

In [6]:
from datasets import load_dataset, load_metric
import pandas as pd

In [7]:
datasets = load_dataset("super_glue", "copa")

datasets

Downloading:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset super_glue downloaded and prepared to /root/.cache/huggingface/datasets/super_glue/copa/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 500
    })
})

In [ ]:
df = pd.DataFrame(datasets["train"])
df.head(10)

,premise,choice1,choice2,question,idx,label
0,My body cast a shadow over the grass.,The sun was rising.,The grass was cut.,cause,0,0
1,The woman tolerated her friend's difficult beh...,The woman knew her friend was going through a ...,The woman felt that her friend took advantage ...,cause,1,0
2,The women met for coffee.,The cafe reopened in a new location.,They wanted to catch up with each other.,cause,2,1
3,The runner wore shorts.,The forecast predicted high temperatures.,She planned to run along the beach.,cause,3,0
4,The guests of the party hid behind the couch.,It was a surprise party.,It was a birthday party.,cause,4,0
5,The politician lost the election.,He ran negative campaign ads.,No one voted for him.,cause,5,1
6,The stain came out of the shirt.,I patched the shirt.,I bleached the shirt.,cause,6,1
7,The man got a discount on his groceries.,He greeted the cashier.,He used a coupon.,cause,7,1
8,The physician misdiagnosed the patient.,The patient filed a malpractice lawsuit agains...,The patient disclosed confidential information...,effect,8,0
9,The customer filed a complaint with the store ...,The sales associate undercharged the customer.,The sales associate acted rude to the customer.,cause,9,1


Make sure that there are only two types of questions (cause and effect), as we will hardcode their treatment in the next section.

In [ ]:
df.question.unique()

array(['cause', 'effect'], dtype=object)

## Preprocessing the data

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [9]:
ending_names = ["choice1", "choice2"]

# Change cause and effect to a proper question
QUESTION_MAP = {
    "cause": "What was the cause of this?",
    "effect": "What happened as a RESULT?",
}

def preprocess_function(examples):
    # Repeat premise and question twice for both possible answers
    first_sentences = [[f"{context} {QUESTION_MAP[question]}"] * 2 for context, question in zip(examples["premise"], examples["question"])]
    # Grab all second sentences possible for each context.
    second_sentences = [[c1, c2] for c1, c2 in zip(examples["choice1"], examples["choice2"])]
    
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    if not len(first_sentences) == len(second_sentences):
        raise ValueError("lengths dont match")
    
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (4) and a list of input IDs (length varying here since we did not apply any padding):

### Test Tokenizer and Preprocessing

In [10]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
examples = datasets["train"][:5]
features = preprocess_function(examples)

print(features.keys())
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

To check we didn't do anything group when grouping all possibilites then unflattening, let's have a look at the decoded inputs for a given example:

In [ ]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(2)]

['[CLS] the runner wore shorts. what was the cause of this? [SEP] the forecast predicted high temperatures. [SEP]',
 '[CLS] the runner wore shorts. what was the cause of this? [SEP] she planned to run along the beach. [SEP]']

We can compare it to the ground truth:

In [23]:
datasets["train"][3]

{'choice1': 'The forecast predicted high temperatures.',
 'choice2': 'She planned to run along the beach.',
 'idx': 3,
 'label': 0,
 'premise': 'The runner wore shorts.',
 'question': 'cause'}

### Apply Preprocessing to the Whole Dataset

In [11]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning the model

In [12]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
import numpy as np

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-copa",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch


def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

Download file pytorch_model.bin:   0%|          | 3.47k/418M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

When called on a list of examples, it will flatten all the inputs/attentions masks etc. in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * 4) x seq_length`) that we then unflatten.

### Test Collator

We can check this data collator works on a list of features, we just have to make sure to remove all features that are not inputs accepted by our model (something the `Trainer` will do automatically for us after):

In [13]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

Again, all those flatten/un-flatten are sources of potential errors so let's make another sanity check on our inputs:

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(2)]

['[CLS] the physician misdiagnosed the patient. what happened as a result? [SEP] the patient filed a malpractice lawsuit against the physician. [SEP] [PAD]',
 '[CLS] the physician misdiagnosed the patient. what happened as a result? [SEP] the patient disclosed confidential information to the physician. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [ ]:
datasets["train"][8]

{'choice1': 'The patient filed a malpractice lawsuit against the physician.',
 'choice2': 'The patient disclosed confidential information to the physician.',
 'idx': 8,
 'label': 0,
 'premise': 'The physician misdiagnosed the patient.',
 'question': 'effect'}

### Run Training

In [14]:
trainer.train()

#model.push_to_hub("felixbmuller/bert-base-uncased-finetuned-copa")

The following columns in the training set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: question, premise, idx, choice2, choice1.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 75


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.685603,0.660000
2,No log,0.603439,0.680000
3,No log,0.564924,0.700000


The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: question, premise, idx, choice2, choice1.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: question, premise, idx, choice2, choice1.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: question, premise, idx, choice2, choice1.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=75, training_loss=0.5885267130533854, metrics={'train_runtime': 42.0252, 'train_samples_per_second': 28.554, 'train_steps_per_second': 1.785, 'total_flos': 36391221099456.0, 'train_loss': 0.5885267130533854, 'epoch': 3.0})

## Evalute the Model


In [15]:
predictions, label_ids, metrics = trainer.predict(encoded_datasets["validation"], metric_key_prefix="val")

The following columns in the test set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: question, premise, idx, choice2, choice1.
***** Running Prediction *****
  Num examples = 100
  Batch size = 16


In [16]:
metrics

{'val_accuracy': 0.699999988079071,
 'val_loss': 0.5649239420890808,
 'val_runtime': 1.0877,
 'val_samples_per_second': 91.937,
 'val_steps_per_second': 6.436}

In [18]:
val = pd.DataFrame(datasets["validation"])
val["label_ids"] = label_ids
val["pred0"] = predictions[:, 0]
val["pred1"] = predictions[:, 1]
val["pred_label"] = np.argmax(predictions, axis=1)

Sanity check to ensure that predictions work the way I expect them to do

In [19]:
print("accuracy", (val.pred_label == val.label).astype(np.float32).mean())

accuracy 0.699999988079071


In [20]:
from google.colab import drive
import joblib
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
joblib.dump(val, "/content/drive/MyDrive/nlp-commonsense/bert-case-uncased-finetuned-copa-baseline-validation-results.joblib")

['/content/drive/MyDrive/nlp-commonsense/bert-case-uncased-finetuned-copa-baseline-validation-results.joblib']

In [22]:
val.head(20)

,premise,choice1,choice2,question,idx,label,label_ids,pred0,pred1,pred_label
0,The man turned on the faucet.,The toilet filled with water.,Water flowed from the spout.,effect,0,1,1,-1.237205,-0.274776,1
1,The girl found a bug in her cereal.,She poured milk in the bowl.,She lost her appetite.,effect,1,1,1,-1.800768,-0.343731,1
2,The woman retired.,She received her pension.,She paid off her mortgage.,effect,2,0,0,-0.213088,-0.727422,0
3,I wanted to conserve energy.,I swept the floor in the unoccupied room.,I shut off the light in the unoccupied room.,effect,3,1,1,-0.797951,-0.893255,0
4,The hamburger meat browned.,The cook froze it.,The cook grilled it.,cause,4,1,1,-0.440235,-1.341200,0
5,I doubted the salesman's pitch.,I turned his offer down.,He persuaded me to buy the product.,effect,5,0,0,-0.391184,-0.870926,0
6,I decided to stay home for the night.,The forecast called for storms.,My friends urged me to go out.,cause,6,0,0,-0.398053,-1.155150,0
7,My eyes became red and puffy.,I was sobbing.,I was laughing.,cause,7,0,0,-0.324752,-1.920901,0
8,The flame on the candle went out.,I blew on the wick.,I put a match to the wick.,cause,8,0,0,-0.690255,-0.575298,1
9,The man drank heavily at the party.,He had a headache the next day.,He had a runny nose the next day.,effect,9,0,0,-1.939929,-1.626201,1


In [24]:
wrong_samples = val[val.label !=  val.pred_label]
wrong_samples.sample(25, random_state=42)

,premise,choice1,choice2,question,idx,label,label_ids,pred0,pred1,pred_label
91,I spent the day at the pool.,I sprained my ankle.,My face got sunburned.,effect,91,1,1,-1.006521,-2.164256,0
38,The bride got cold feet before the wedding.,The wedding guests brought gifts.,She called the wedding off.,effect,38,1,1,-0.736364,-1.442032,0
73,The seasons changed from summer to autumn.,People evacuated their homes.,Leaves fell from the trees.,effect,73,1,1,-1.567018,-1.573198,0
49,I pressed my hand into the wet cement.,My handprint dried in the cement.,Cracks emerged in the cement.,effect,49,0,0,-0.059384,0.293413,1
25,The driver got a flat tire.,He went over the speed limit.,He ran over a nail.,cause,25,1,1,0.153390,-1.071954,0
26,My view of the movie screen was blocked.,The couple behind me was whispering.,A tall person was sitting in front of me.,cause,26,1,1,-0.323148,-0.654481,0
97,The airline mishandled my luggage.,They offered me compensation.,They cancelled my flight.,effect,97,0,0,-0.207793,0.014523,1
82,The jewelry thieves were caught.,The stolen jewelry was returned to its owners.,The cost of the stolen jewelry was calculated.,effect,82,0,0,0.135179,0.236171,1
35,The police searched the offender's car.,They were trying to elicit a confession.,They were looking for illegal drugs.,cause,35,1,1,-0.158606,-0.635507,0
3,I wanted to conserve energy.,I swept the floor in the unoccupied room.,I shut off the light in the unoccupied room.,effect,3,1,1,-0.797951,-0.893255,0
